In [168]:
from pathlib import Path
Home = str(Path.home())

import sys
path = f'{Home}/repos/EPObjects/'
sys.path.insert(0, path)

import numpy as np

from IDFObject.BuildingSurface.Detailed import Detailed as BS
from IDFObject.FenestrationSurface.Detailed import Detailed as FS
from IDFObject.Zone import Zone
from IDFObject.ZoneList import ZoneList

from IDFObject.Construction import Construction
from GeometryObject.XYZList import XYZList


In [169]:
surface_match_dict = ['Floor', 'Ceiling', 'Wall', 'Roof']
def match_surface(i):
    return surface_match_dict[i]

construction_match_list = {
    'GroundFloor': Construction.FloorGround['Name'],
    'ExternalWall': Construction.WallExternal['Name'],
    'Glazing': Construction.Glazing['Name'],
    'Floor_Ceiling': Construction.FloorInternal['Name'],
    'InternalWall': Construction.WallInternal['Name'],
    'Roof': Construction.Roof['Name'],
}

def match_construction(i):
    return construction_match_list[i]

def convertToStrRepr(xyzList):
    points = list()
    try:
        for ps in xyzList:
            for p in xyzList[ps]:
                points += [[p["X"],p["Y"],p["Z"]]]
    except KeyError:
        for p in xyzList['XYZs']:
            points += [[p["X"],p["Y"],p["Z"]]]

    return str(XYZList(np.array(points)))

def wrap(elem, **kwargs):
    func = kwargs.get('func', None)
    prop = kwargs.get('prop', None)
    return func(elem[prop]) if func else elem[prop] if prop else None

mappings = {
    'BuildingSurface:Detailed': {
        'SpaceName': {},
        'XYZs': {'func': convertToStrRepr, 'prop': 'XYZList'},
        'OutsideBoundaryCondition': {'prop': 'OutsideCondition'},
        'OutsideBoundaryConditionObject': {'prop': 'OutsideObject'},
        'SunExposure': {'prop': 'SunExposed'},
        'WindExposure': {'prop': 'WindExposed'},
        'ViewFactor': {},
        'SurfaceType': {'func': match_surface, 'prop': 'SurfaceType'},
        'ConstructionName': {'func': match_construction, 'prop': 'ConstructionName'},
    },

    'FenestrationSurface:Detailed': {
        'SpaceName': {},
        'BuildingSurfaceName': {'prop': 'FaceName'},
        'XYZs': {'func': convertToStrRepr, 'prop': 'VerticesList'},
        'OutsideBoundaryConditionObject': {},
        'FrameAndDividerName': {},
        'Multiplier': {},
        'ViewFactor': {},
        'ConstructionName': {'func': match_construction, 'prop': 'ConstructionName'},
    },
}

In [174]:
def convert_data(data):
    for b in data:
        json_elements = []

        zone_names = []
        for z in b['zones']:
            for s in z['Surfaces']:
                mapping = mappings['BuildingSurface:Detailed']     
                ns = dict(s)
                for key in mapping:
                    ns[key] = wrap(s, **mapping[key])

                json_elements.append(BS(ns))

                if 'Fenestrations' in s.keys() and s['Fenestrations']:
                    for f in s['Fenestrations']:
                        mapping = mappings['FenestrationSurface:Detailed']     
                        nf = dict(f)
                        for key in mapping:
                            nf[key] = wrap(f, **mapping[key])
                        
                        json_elements.append(FS(nf))
                # break
            # break
            zone_names.append(z['Name'])
        
        for z in zone_names:
            json_elements.append(Zone({'Name': z}))
        
        json_elements.append(ZoneList({"Name": 'Office', "ZoneNames": ','.join(zone_names)}))
        yield b["name"], json_elements
    

In [189]:
import json
import requests
from IDFObject.IDFObject import IDFJsonEncoder

client = requests.session()
URL = "http://127.0.0.1:3001/"

for i in range(12):
    with open(f'Data/Buildings_{i:03}.json') as f:
        data = json.load(f)

        for name, building in convert_data(data):
            geom_str = json.dumps(building, cls=IDFJsonEncoder)
            query = {
                "TYPE": "INSERT_ITEM", 
                "TABLE_NAME": "PROJECTS",
                "COLUMN_NAMES": "(PROJECT_NAME,USER_NAME,GEOMETRY)",
                "COLUMN_VALUES": f"('{name}','RANDOM','{geom_str}')",
            }
            response = requests.post(URL, json=query).json()

            if (response["ERROR"]):
                print (response['QUERY'])
                raise Exception(response["ERROR"])

In [205]:
query = {
            "TYPE": "SEARCH", 
            "TABLE_NAME": "PROJECTS",
            "COLUMN_NAMES": "PROJECT_NAME,USER_NAME",
            "CONDITIONS": f"USER_NAME='RANDOM'",
        }
response = requests.post(URL, json=query).json()
print (len(response['RESULT']))

1200
